In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xtiff

from deepcell.applications import Mesmer
from matplotlib.colors import ListedColormap
from pathlib import Path
from skimage.segmentation import expand_labels
import sklearn

from steinbock import io
from steinbock.preprocessing import imc
from steinbock.segmentation import deepcell
from steinbock.measurement import intensities, regionprops, neighbors

import helpers

# IMC preprocessing pipeline

This pipeline will extract image data from IMC aquisitions located in the `raw` folder and generate single cell data. Intermediary steps include the extraction of image stacks selected from the panel file as well as segmentation masks. The single cell data is extracted from the chanels selected in the panel.

Before running your own script please check the [steinbock documentation](https://bodenmillergroup.github.io/steinbock).

*Installation*  
To install the required python environment, follow the instructions here: https://bodenmillergroup.github.io/steinbock/latest/install-python/

## Settings

Example data can be downloaded using the `download_examples.ipynb` script.

### Input and output directories

Folder structure:

In [ ]:
resolution = 'low_res' #high_res=4000x4000, low_res=700x700
modality = 'IF'
img_folder = 'img_' + resolution
parameter_set = 'parameter_set1'

sup_super_base = Path("/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Segmentation/20220723_Mesmer_IFvsIMC")

# Output directories
img_dir = sup_super_base / img_folder
super_base = sup_super_base / parameter_set
summary_dir = super_base /"summary"
base_dir_top = super_base / resolution
base_dir = base_dir_top / modality
masks_dir = base_dir / "masks"
segstack_dir = base_dir / "segstacks"
outline_dir = masks_dir / 'outline'

# Create directories (if they do not already exist)
#super_base.mkdir(exist_ok=True)
super_base.mkdir(exist_ok=True)
base_dir_top.mkdir(exist_ok=True)
base_dir.mkdir(exist_ok=True)
summary_dir.mkdir(exist_ok=True)
img_dir.mkdir(exist_ok=True)
masks_dir.mkdir(exist_ok=True)
segstack_dir.mkdir(exist_ok=True)
outline_dir.mkdir(exist_ok=True)

## Extract images from `.mcd` files

Documentation: https://bodenmillergroup.github.io/steinbock/latest/cli/preprocessing/#external-images

### Import the panel
The antibody panel should meet the steinbock format: https://bodenmillergroup.github.io/steinbock/latest/file-types/#panel  

An example panel corresponding to the example data (downloadable via the `download_examples.ipynb` script is provided.

Customized panels should contain the following columns:
+ `channel`: unique channel id, typically metal and isotope mass (e.g. `Ir191`)
+ `name`: unique channel name.
+ `deepcell`: channels to use for segmentation (1=nuclear, 2=membrane, empty/NaN=ignored).
+ `keep`: *(optional)* 1 for channels to preprocess, 0 for channels to ignore

In [ ]:
imc_panel = pd.read_csv('panel_' + modality + '.csv')

if "keep" in imc_panel.columns:
    imc_panel = imc_panel[imc_panel["keep"]==1]
imc_panel.head()

## Cell segmentation

Documentation: https://bodenmillergroup.github.io/steinbock/latest/cli/segmentation/#deepcell  

### Prepare segmentation stacks

Segmentation stacks are generated by aggregating the channels selcted in `panel.csv` in the column `deepcell`. 
Cell segmentation requires to construct as 2-channel images with the following structure:
+ Channel 1 = nuclear channels
+ Channel 2 = cytoplasmic/membranous channels.

For channel-wise normalization, zscore and min-max methods are available.  
In addition, different functions can be used to aggregate channels. Default: `np.mean`, for other options, see https://numpy.org/doc/stable/reference/routines.statistics.html#averages-and-variances.

In [ ]:
# Define image preprocessing options
channelwise_zscore = True
channelwise_minmax = False
aggr_func = np.sum

# Define channels to use for segmentation (from the panel file)
channel_groups = imc_panel["deepcell"].values
channel_groups = np.where(channel_groups == 0, np.nan, channel_groups) # make sure unselected chanels are set to nan

#### Generate segmentation stacks

In [ ]:
for img_path in sorted(Path(img_dir).glob("*.tiff")):
    img = io.read_image(img_path)
    if channelwise_minmax:
        img = helpers.norm_minmax(img)
    
    if channelwise_zscore:
        img = helpers.norm_zscore(img)
    
    if channel_groups is not None:
        img = helpers.segstack_channels(img, channel_groups, aggr_func)
    
    img_file = Path(segstack_dir) / f"{img_path.name}"
    io.write_image(img, img_file)

#### Check segmentation stacks

In [ ]:
# List segmentation stacks
segstacks = sorted(Path(segstack_dir).glob("*.tiff"))

# Select a random image
rng = np.random.default_rng()
ix = rng.choice(len(segstacks))

# Display nuclear and membrane/cytoplasm images
fig, ax = plt.subplots(1, 2, figsize=(30, 30))

img = io.read_image(segstacks[ix])
ax[0].imshow(img[0,:,:], vmin=0, vmax=8) # adjust vmax if needed (lower value = higher intensity)
ax[0].set_title(segstacks[ix].stem + ": nuclei")

img = io.read_image(segstacks[ix])
ax[1].imshow(img[1,:,:], vmin=0, vmax=8) # adjust vmax if needed (lower value = higher intensity)
ax[1].set_title(segstacks[ix].stem + ": membrane")

### Segment cells

`segmentation_type` should be one of [`whole-cell`, `nuclear`, `both`].  
If `both` is selected, nuclear and whole-cell masks will be generated in separate subfolders.  

Several post-processing arguments can be passed to the deepcell application, the defaults are selected below. Cell labels can also be expanded by defining an `expansion_distance` (mostly useful for nuclear segmentation).

In [ ]:
# Segmentation type
segmentation_type = "nuclear"

# Post-processing arguments for whole-cell segmentation
kwargs_whole_cell =  {
    'maxima_threshold': 0.075,
    'maxima_smooth': 0,
    'interior_threshold': 0.2,
    'interior_smooth': 2,
    'small_objects_threshold': 15,
    'fill_holes_threshold': 15,
    'radius': 2
}

# Post-processing arguments for nuclear segmentation
kwargs_nuclear =  {
    'maxima_threshold': 0.1,
    'maxima_smooth': 0,
    'interior_threshold': 0.2,
    'interior_smooth': 2,
    'small_objects_threshold': 15,
    'fill_holes_threshold': 15,
    'radius': 2
}

df = pd.DataFrame(kwargs_nuclear, index=[0])
df.to_csv(super_base / 'kwargs_nuclear.csv', sep=';')

# Mask pixel expansion (0 = no expansion)
expansion_distance = 0

#Micrometer per pixel
if resolution == 'high':
    mpp=0.2
else:
    mpp=1.0

In [ ]:
from deepcell.utils.plot_utils import make_outline_overlay, create_rgb_image
from skimage.segmentation import find_boundaries
import tifffile as tiff

def make_outline_overlay(rgb_data, predictions):

    boundaries = np.zeros_like(rgb_data)
    overlay_data = np.copy(rgb_data)

    for img in range(predictions.shape[0]):
        boundary = find_boundaries(predictions[img, ..., 0], connectivity=1, mode='thick')
        boundaries[img, boundary > 0, :] = 1

    overlay_data[boundaries > 0] = 1

    return overlay_data

def create_outline(im, mask):
    
    rgb = create_rgb_image(im, ["blue"])
    outline = make_outline_overlay(rgb, mask)
    
    return outline.squeeze()

if modality == 'IF':
    idx = 1
else:
    idx=0

app = Mesmer()

for stack in segstacks:
    img = io.read_image(stack)
    img = np.moveaxis(img, 0, 2)
    img = np.expand_dims(img.data, 0)
    
    mask = app.predict(
        img, image_mpp=mpp, compartment=segmentation_type,
        postprocess_kwargs_whole_cell=kwargs_whole_cell,
        postprocess_kwargs_nuclear=kwargs_nuclear
    )

    tmp_img = np.zeros_like(img)
    tmp_img = np.expand_dims(tmp_img[:,:,:,idx], -1)
    
    o = create_outline(tmp_img, mask)

    img_name = stack.name.split('.')[0] + '_' + modality + '_img.tif'
    mask_name = stack.name.split('.')[0] + '_' + modality + '_mask.tif'
    raw = tiff.imread(img_dir / stack.name)
    tiff.imwrite(outline_dir/ img_name, raw[idx])
    tiff.imwrite(outline_dir / mask_name, o[:,:,0])
    
    helpers.save_masks(
        mask, masks_dir, stack.name,
        segmentation_type, expansion_distance
    )

In [ ]:
import shutil
import os

files = os.listdir(outline_dir)
 
shutil.copytree(outline_dir, summary_dir, dirs_exist_ok=True)